In [1]:
from mmcv import Config
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pycocotools.coco import COCO

In [2]:
model_cfg = \
"detectors_cascade_rcnn_r50_1x_coco"

# load config.
cfg = Config.fromfile(os.path.join("save_dir", model_cfg, "configs.py"))

In [3]:
# checkpoint
checkpoint_path = os.path.join(cfg.work_dir, "best_bbox_mAP_50.pth")

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=cfg.data.workers_per_gpu, dist=False, shuffle=False)

model = build_detector(cfg.model, test_cfg=cfg.get("test_cfg"))
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2021-05-20 05:08:30,244 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-20 05:08:30,245 - mmdet - INFO - Use load_from_torchvision loader
2021-05-20 05:08:30,496 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

Use load_from_local loader


In [4]:
output = single_gpu_test(model, data_loader, show_score_thr=0.001)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.8 task/s, elapsed: 1081s, ETA:     0s

In [5]:
try:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
except:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[0][j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f"submission_{model_cfg}.csv"), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.15096433 221.43513 255.63554 276.79327 294...,batch_01_vt/0021.jpg
1,1 0.37431994 0.6438844 244.75012 302.58282 457...,batch_01_vt/0028.jpg
2,1 0.9968338 73.05599 168.46013 374.11765 379.1...,batch_01_vt/0031.jpg
3,1 0.11087761 296.0267 156.57527 358.55698 224....,batch_01_vt/0032.jpg
4,1 0.74530506 388.31796 270.92026 479.56686 343...,batch_01_vt/0070.jpg
